## بخش اول از جمع آوری داده ها با استفاده از راهنما و رابط تعریف شده سایت پروژه اسپیس ایکس 

In [ ]:
# فراخوانی کتاب خانه های مورد نیاز
import requests
import pandas as pd
import numpy as np
import datetime
# تنظیمات برای نمایش همه ستون ها
pd.set_option('display.max_columns', None)
# تنظیمات برای نمایش همه ستون ها
pd.set_option('display.max_colwidth', None)

In [ ]:
# دریافت داده ها از سایت پروژه
spacex_url="https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)
#response.status_code
#print(response.content)

# تبدیل فایل جیسون به دیتا فریم
data = response.json()
data = pd.json_normalize(data)
#data.head()

# جدا کردن  ویژگی های مورد نیاز
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# حذف سطر های حاوی چند حالت
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

#خارج کردن متغیر ها از حالت لیست 
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# تعریف متغییر تاریخ
data['date'] = pd.to_datetime(data['date_utc']).dt.date

#  ایجاد محدودیت بر اساس تاریخ پرتاب
data = data[data['date'] <= datetime.date(2020, 11, 13)]

#data.head()

In [ ]:
# لیست متغییر های مورد نیاز که باید استخراج شود
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [ ]:
# تابع کال کردن اسم راکت با استفاده از کد مربوطه
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])


# تابع کال اسم لانچ پد و مختصات جغرافیایی با استفاده از کد مربوطه
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])


# تابع کال وزن پرتابه و اوربیت
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])


# تابع تفکیک متغیر کور و اضافه کردن مقادیر آن به لیست های مربوطه
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [ ]:
# اجرای توابع 
getBoosterVersion(data)

getLaunchSite(data)

getPayloadData(data)

getCoreData(data)

#BoosterVersion[0:5]

In [ ]:
# ادغام لیست ها و ساخت دیکشنری برای داده های استخراج شده
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

# تبدیل دیکشنری به دیتا فریم
data = pd.DataFrame.from_dict(launch_dict)
data.head()

In [ ]:
# حذف داده های مربوط به فالکون یک از داده ها
data_falcon9 = data[data['BoosterVersion']!="Falcon 1"]

# بعد از حذف داده های فالکون یک مجدد شماره پرتاب ها ست می شود
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

In [ ]:
# بررسی میسینگ ها
data_falcon9.isnull().sum()

In [ ]:
# محاسبه میانگین ستون پیلود
mean_PayloadMass = data_falcon9["PayloadMass"].mean()

# جای گذاری مقادیر گم شده با مقدار میانگین
data_falcon9["PayloadMass"].replace(np.nan, mean_PayloadMass, inplace= True)

In [ ]:
# خروجی گرفتن از دادها
data_falcon9.to_csv('dataset_part_1.csv', index=False)